### Time travel
The goal of this notebook is to add the ability to select data based on how long it has existed.

We'd ideally like to do this in two ways - either by selecting the data release version or by selecting the amount of time between the exposure and now. This is useful if you want to only send data to Zooniverse that is not proprietary (greater than 2 years old).

Here's the forum that talks about how to get MJD info from the Rubin source catalog:
https://community.lsst.org/t/how-should-one-efficiently-get-the-mjd-for-a-source-catalog/3733/4

In [10]:
# Generic python packages
import pylab as plt
import gc

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst
from datetime import datetime
from astropy.time import Time

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

In [2]:
config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collections)

In [3]:
datasetType = 'calexp'
dataId = {'visit': 192350, 'detector': 175}
calexp = butler.get(datasetType, dataId=dataId)

In [13]:
# This is the time of the observation in MJD:
t1 = calexp.getInfo().getVisitInfo().getDate().get(lsst.daf.base.DateTime.MJD)
print(t1)
# You can also get UTC:
#print(calexp.getInfo().getVisitInfo().getDate().get(lsst.daf.base.DateTime.MJD,lsst.daf.base.DateTime.UTC))


59837.38009422338
59837.37966598263


### Below, finding the difference between 'now' and the observation:
To do: Is there a way to improve this (take away the 2*365 option)?

In [9]:
# Now get the time of right now and see their difference:

nt = Time.now()
#ut = Time(datetime.utcnow())



if nt.mjd - t1 > 2*365:
    print(f'Obs is older than 2 years {round(nt.mjd - t1,1)} days old')
else:
    print(f'Obs is younger than 2 years {round(nt.mjd - t1,1)} days old')

Obs is younger than 2 years 61.5 days old


In [47]:
print(nt.mjd - 2*365)

59168.91007075441


### Is there a way to modify the below query to include a time restriction?


In [14]:
# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

import lsst.daf.butler as dafButler
import lsst.geom
import lsst.afw.display as afwDisplay

plt.style.use('tableau-colorblind10')
%matplotlib inline

import warnings
from astropy.units import UnitsWarning
import pandas

#initializing Tap and Butler
pandas.set_option('display.max_rows', 20)
warnings.simplefilter("ignore", category=UnitsWarning)
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

In [15]:
max_rec=10 # make 100 for full subject set test
use_center_coords = "62, -37"
use_radius = "1.0"

In [20]:
### Modify the query

In [33]:
query = "SELECT TOP " + str(max_rec) + " " + \
        "obj.objectId, obj.coord_ra, obj.coord_dec, obj.detect_isPrimary " + \
        "g_cModelFlux, r_cModelFlux, obj.r_extendedness, obj.r_inputCount " + \
        "FROM dp02_dc2_catalogs.Object as obj " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND r_extendedness = 1 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 18.0 " + \
        "ORDER by r_cModelFlux DESC"
results = service.search(query)
print(results)

<Table length=10>
      objectId       coord_ra   coord_dec  ... r_extendedness r_inputCount
                       deg         deg     ...                            
       int64         float64     float64   ...    float64        int32    
------------------- ---------- ----------- ... -------------- ------------
1567965153859768169 61.6998044 -37.2739044 ...            1.0          106
1650947495431285770 61.1583878 -36.3657651 ...            1.0          105
1651448872733547971 62.5770683 -36.1977335 ...            1.0          116
1651536833663756158 61.8484527 -36.1061415 ...            1.0          105
1651325727431231924 62.7137842 -36.6836861 ...            1.0          117
1651334523524249766 62.3059651 -36.7486842 ...            1.0          110
1650894718873158112 60.8890681 -36.5646567 ...            1.0          107
1651466464919580824 62.1611157  -36.333154 ...            1.0          107
1651325727431231922 62.6921133 -36.6985965 ...            1.0          112
1651528

In [ ]:
query = "SELECT TOP " + str(max_rec) + " " + \
        "obj.objectId, obj.coord_ra, obj.coord_dec, obj.detect_isPrimary " + \
        "g_cModelFlux, r_cModelFlux, obj.r_extendedness, obj.r_inputCount " + \
        "FROM dp02_dc2_catalogs.Object as obj " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND r_extendedness = 1 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 18.0 " + \
        "ORDER by r_cModelFlux DESC"
results = service.search(query)
print(results)

In [44]:
query = "SELECT TOP " + str(max_rec) + " " + \
        "obj.objectId, obj.coord_ra, obj.coord_dec, obj.detect_isPrimary " + \
        "g_cModelFlux, r_cModelFlux, obj.r_extendedness, obj.r_inputCount, ccd.obsStartMJD " + \
        "FROM dp02_dc2_catalogs.Object as obj " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND r_extendedness = 1 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 18.0 " + \
        "ORDER by r_cModelFlux DESC"
results = service.search(query)

assert len(results) == max_rec

DALQueryError: validateColumnNonAlias: Column: [obsStartMJD] does not exist.

In [22]:
query = "SELECT TOP " + str(max_rec) + " " + \
        "obj.objectId, obj.coord_ra, obj.coord_dec, obj.detect_isPrimary " + \
        "obj.g_cModelFlux, obj.r_cModelFlux, obj.r_extendedness, obj.r_inputCount, ccd.obsStartMJD " + \
        "FROM dp02_dc2_catalogs.Object as obj " + \
        "JOIN dp02_dc2_catalogs.CcdVisit as ccd " + \
        "ON obj.ccdVisitId = ccd.ccdVisitId " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND r_extendedness = 1 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 18.0 " + \
        "ORDER by r_cModelFlux DESC"
results = service.search(query)

assert len(results) == max_rec

DALQueryError: ADQL syntax error: Encountered " "." ". "" at line 1, column 82.
Was expecting one of:
    "INTO" ...
    "FROM" ...
    "," ...
    "," ...
    "INTO" ...
    "FROM" ...

In [49]:
query = "SELECT fs.forcedSourceId, fs.objectId, fs.ccdVisitId, fs.detect_isPrimary, " + \
        "fs.band, scisql_nanojanskyToAbMag(fs.psfFlux) as psfMag, ccd.obsStartMJD, " + \
        "scisql_nanojanskyToAbMag(obj.r_psfFlux) as obj_rpsfMag " + \
        "FROM dp02_dc2_catalogs.ForcedSource as fs " + \
        "JOIN dp02_dc2_catalogs.CcdVisit as ccd " + \
        "ON fs.ccdVisitId = ccd.ccdVisitId " + \
        "JOIN dp02_dc2_catalogs.Object as obj " + \
        "ON fs.objectId = obj.objectId " + \
        "WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND obj.detect_isPrimary = 1 " + \
        "AND obj.r_extendedness = 0 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) > 17.5 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) < 18.0 " + \
        "AND fs.band = 'r' "
results = service.search(query)
print(results)

<Table length=41751>
  forcedSourceId         objectId      ...  obsStartMJD     obj_rpsfMag    
      int64               int64        ...    float64         float64      
------------------ ------------------- ... ------------- ------------------
161196576881003935 1650885922780118031 ... 60001.0762086 17.775600417632916
654224497083167651 1650885922780118031 ... 61373.2747466 17.775600417632916
130242261710570697 1650885922780118031 ... 59914.1938026 17.775600417632916
640882147355297539 1650885922780118031 ... 61329.1591576 17.775600417632916
423520334147251033 1650885922780118031 ... 60740.0680676 17.775600417632916
384442609101212165 1650885922780118031 ... 60636.1544946 17.775600417632916
520791108494572004 1650885922780118031 ... 61007.2110296 17.775600417632916
234387686339674243 1650885922780118031 ... 60206.3064806 17.775600417632916
108779734069957865 1650885922780118031 ... 59854.2944256 17.775600417632916
540103432699152173 1650885922780118031 ... 61066.0683766 17.7756004

In [53]:
nt.mjd - 50

59848.91007075441

In [55]:
query = "SELECT fs.forcedSourceId, fs.objectId, fs.ccdVisitId, fs.detect_isPrimary, " + \
        "fs.band, scisql_nanojanskyToAbMag(fs.psfFlux) as psfMag, ccd.obsStartMJD, " + \
        "scisql_nanojanskyToAbMag(obj.r_psfFlux) as obj_rpsfMag " + \
        "FROM dp02_dc2_catalogs.ForcedSource as fs " + \
        "JOIN dp02_dc2_catalogs.CcdVisit as ccd " + \
        "ON fs.ccdVisitId = ccd.ccdVisitId " + \
        "JOIN dp02_dc2_catalogs.Object as obj " + \
        "ON fs.objectId = obj.objectId " + \
        "WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND obj.detect_isPrimary = 1 " + \
        "AND obj.r_extendedness = 0 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) > 17.5 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) < 18.0 " + \
        "AND fs.band = 'r' " + \
        f"AND ccd.obsStartMJD < {nt.mjd - 50} "
results = service.search(query)
print(results)

<Table length=2909>
  forcedSourceId         objectId      ...  obsStartMJD     obj_rpsfMag    
      int64               int64        ...    float64         float64      
------------------ ------------------- ... ------------- ------------------
104088585568802545 1567921173394650080 ... 59840.2628446  17.84863392562694
103716499667057546 1567921173394650080 ... 59839.3528966  17.84863392562694
104071441133114408 1567921173394650080 ... 59840.2469706  17.84863392562694
104047259930363137 1567921173394650080 ... 59840.2231276  17.84863392562694
 21649872503473461 1567921173394650080 ... 59634.0864666  17.84863392562694
103714346277831446 1567921173394650080 ... 59839.3510616  17.84863392562694
104064947679422127 1567921173394650080 ... 59840.2414956  17.84863392562694
103695564922713720 1567921173394650080 ... 59839.3343466  17.84863392562694
 21650458229624109 1567921173394650080 ... 59634.0869156  17.84863392562694
103714346277822435 1567859600743513720 ... 59839.3510616 17.78573960